### Асинхронное программирование

### О чём говорили в прошлый раз?

* Процессы
* Менеджмент процессов
* Многопроцессные приложения на Python
* Multiprocessing (Pool)

### План сегодняшнего занятия

* Потоки
* Многопоточные приложения
* GIL
* Конкуррентность vs параллельность
* Асинхронность
* asyncio

### Поток (thread)

* Малейшая единица программного вычисления.

* Управляется планировщиком задач (scheduler)

* Создаётся быстрее, чем процесс

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Multithreaded_process.svg/330px-Multithreaded_process.svg.png)

### Многопоточность

* Когда программа много "ожидает", i\o bound tasks

* Между потоками общая память

* Потоки могут работать "конкурентно" (concurrently) с возможностью пересечения

In [1]:
import requests
import time
import concurrent.futures

img_urls = [
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
]

t1 = time.perf_counter()


def download_image(img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_image, img_urls)


t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

photo-1532009324734-20a7a5813719.jpg was downloaded...
photo-1516117172878-fd2c41f4a759.jpg was downloaded...
Finished in 0.734096173895523 seconds


### GIL (global interpreter lock)

Только один поток за раз может управлять интерпретатором

![](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1585364510/gil2_kgyuui.png)

### Синхронность vs Асинхронность

Синхронно: по одному за раз

Асинхронно: выполняем следующую задачу, пока выполняется предыдущая

![](https://www.koyeb.com/static/images/blog/sync-vs-async-schema.png)

Пример из шахмат:

https://www.youtube.com/watch?v=iG6fr81xHKA&t=269s

![](https://i.pinimg.com/originals/f6/a1/9c/f6a19cd6a24b111b7968983f3f15e4d0.jpg)

### asyncio

* async \ await

* coroutines (корутины)

* event loop

Команда __await__ возвращает контроль циклу событий (event loop)

In [ ]:
async def g():
    r = await f() # Pause here and come back to g() when f() is ready
    return r


Основные вещи, которые важно понять

* Как создавать асинхронные задачи (корутины, которые могут вернуть контроль циклу событий)

* Как создавать цикл событий

* Как подавать задачи циклу событий

* Как запускать цикл событий

In [6]:
async def read_lecture(sleep_time):
    print("hello")
    await asyncio.sleep(sleep_time)
    print("...saying boring stuff...")
    await asyncio.sleep(sleep_time)
    print("done!")
    return "knowledge"

# read_lecture(1) <-- coroutine object

In [ ]:
loop = asyncio.get_event_loop()
task_one = loop.create_task(read_lecture(1))
loop.run_until_complete()

In [ ]:
async def read_lecture(sleep_time):
    print("hello")
    await asyncio.sleep(sleep_time)
    print("...saying boring stuff...")
    await asyncio.sleep(sleep_time)
    print("done!")
    return "knowledge"

In [ ]:
async def drink_water(sleep_time):
    print("first sip")
    await asyncio.sleep(sleep_time)
    print("second_sip")
    return "hydrated!"



In [ ]:
loop = asyncio.get_event_loop()
task_one = loop.create_task(read_lecture(1))
task_two = loop.create_task(drink_water(1))

a = loop.run_until_complete(task_two)
print(a)

In [ ]:
b = loop.run_until_complete(task_one)
print(b)

In [ ]:
loop = asyncio.get_event_loop()
task_one = loop.create_task(read_lecture(1))
task_two = loop.create_task(drink_water(1))
c = loop.run_until_complete(asyncio.gather(task_one, task_two))
print(c)

Основные вещи, которые важно понять

* Как создавать асинхронные задачи (корутины, которые могут вернуть контроль циклу событий)

_async_ перед функцией (async def foo():)

_await_ чтобы вернуть управление циклу событий


* Как создавать цикл событий

loop = asyncio.get_event_loop()

* Как подавать задачи циклу событий

task = loop.create_task(async_function())

* Как запускать цикл событий

loop.run_until_complete(task)

In [4]:
import asyncio

@asyncio.coroutine
def py34_coro():
    """Generator-based coroutine, older syntax"""
    yield from stuff()

async def py35_coro():
    """Native coroutine, modern syntax"""
    await stuff()


/tmp/ipykernel_1070682/61514977.py:4: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def py34_coro():


In [ ]:
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
finally:
    loop.close()


In [ ]:
asyncio.run(main())  # Python 3.7+


In [ ]:
import requests
import time

start_time = time.time()

for number in range(1, 151):
    url = f'https://pokeapi.co/api/v2/pokemon/{number}'
    resp = requests.get(url)
    pokemon = resp.json()
    print(pokemon['name'])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import aiohttp
import asyncio


async def main():

    async with aiohttp.ClientSession() as session:
        # for
        pokemon_url = 'https://pokeapi.co/api/v2/pokemon/151'
        async with session.get(pokemon_url) as resp:
            pokemon = await resp.json()
            print(pokemon['name'])

asyncio.run(main())

In [ ]:
import aiohttp
import asyncio
import time

start_time = time.time()


async def get_pokemon(session, url):
    async with session.get(url) as resp:
        pokemon = await resp.json()
        return pokemon['name']


async def main():

    async with aiohttp.ClientSession() as session:

        tasks = []
        for number in range(1, 151):
            url = f'https://pokeapi.co/api/v2/pokemon/{number}'
            tasks.append(asyncio.ensure_future(get_pokemon(session, url)))

        original_pokemon = await asyncio.gather(*tasks)
        for pokemon in original_pokemon:
            print(pokemon)

asyncio.run(main())
print("--- %s seconds ---" % (time.time() - start_time))
